# Synapse Databricks Streaming Demo

In this notebook, we'll illustrate two examples:

1. Reading a Delta table in Synapse
2. Spark Streaming in Synapse and how it can leverage streams started in Databricks.


In [ ]:
# Example 1: Show how a delta format dataset can be loaded and displayed
container_name = 'synstorage'
storage_account_name = 'strdbwsynworkshop'
df = spark.read.format('delta').load(f'abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/delta_test')
display(df)

Example 2: Delta streaming. 

This example has two streams: 

(a) a readStream from a Delta table in ADLS; and 

(b) a writeStream to a new Delta table in ADLS

Between these two steps, we'll add a small transformation to show how the streams 
can be adjusted.

In [ ]:
# Example 2
container_name = 'synstorage'
storage_account_name = 'strdbwsynworkshop'
delta_streaming_path = f'abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/delta_test_partitioned_stream'
delta_streaming_path_output = f'abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/delta_test_partitioned_stream_synapse'

df_stream = (
    # Create a read stream from delta format
    spark.readStream.format('delta').load(delta_streaming_path)
    # Apply a rename transformation
    .withColumnRenamed('a', 'new_a')
    # Write new stream as a new Delta dataset in ADLS
    .writeStream.format("delta").outputMode("append")
    .option("checkpointLocation", "/delta_test_partitioned_stream_synapse_checkpoint")
    .start(delta_streaming_path_output)
)

In [ ]:
# Stream data cannot be viewed in Synapse, however, we can see the status info
# on the Spark stream.
print(df_stream.isActive)
print(df_stream.status)

In [ ]:
# We stop the stream when we're done with it.
df_stream.stop()
print(df_stream.isActive)